# [`ruamel.yaml`](https://yaml.readthedocs.io/en/latest/index.html)

## [Overview](https://yaml.readthedocs.io/en/latest/overview.html)

## [Install](https://yaml.readthedocs.io/en/latest/install.html)


In [1]:
from __future__ import print_function

import inspect
import os
from pathlib import Path
import sys

import ruamel.yaml
from ruamel.yaml import YAML
from ruamel.yaml import yaml_object
from ruamel.yaml.compat import StringIO

In [2]:
BASE_DIR = Path(os.getcwd())
print(BASE_DIR)

/workspaces/ruamel_yaml/ipynb/ruamel


---

## [Basic Usage](https://yaml.readthedocs.io/en/latest/basicuse.html)

In [3]:
def demo_load():
    """demo_load"""
    doc = """a:\n  b: 2\n  c: 3\n"""

    # C based SafeLoader
    # Note: C yaml library doesn’t generate CommentTokens,
    #  so it cannot be used to do round trip editing on comments.
    yaml_c=YAML(typ='safe')   # default, if not specified, is 'rt' (round-trip)
    yaml_c.load(doc)

    # Python based SafeLoader
    yaml_py=YAML(typ='safe', pure=True)
    yaml_py.load(doc)


if demo_load() is None:
    print("demo_load() = Pass")

demo_load() = Pass


In [4]:
def demo_write():
    """demo_write"""
    f_yml = 'demo_write.yml'

    with open(BASE_DIR / f_yml, 'w', encoding='utf-8') as file_stream:
        yaml=YAML()
        yaml.default_flow_style = False
        yaml.dump({'a': [1, 2]}, file_stream)


if demo_write() is None:
    print("demo_write() = Pass")

demo_write() = Pass


---

## [Dumping Python classes](https://yaml.readthedocs.io/en/latest/dumpcls.html)

In [5]:
def demo_register_class():
    """demo_register_class"""
    class User(object):
        """User"""
        def __init__(self, name, age):
            self.name = name
            self.age = age

    yaml = ruamel.yaml.YAML()
    yaml.register_class(User)
    yaml.dump([User('Anthon', 18)], sys.stdout)


if demo_register_class() is None:
            print("demo_register_class() = Pass")    

- !User
  name: Anthon
  age: 18
demo_register_class() = Pass


In [6]:
def demo_decorator_class():
    """demo_decorator_class
    The yaml_tag, from_yaml and to_yaml work in the same way as
    when using .register_class()
    """
    f_yml = 'demo_decorator_class.yml'
    yaml = YAML()  # decorator, takes YAML() instance as parameter

    @yaml_object(yaml)
    class User:
        """User"""
        yaml_tag = '!User'

        def __init__(self, name, age):
            self.name = name
            self.age = age

        @classmethod
        def to_yaml(cls, representer, node):
            """to_yaml"""
            return representer.represent_scalar(cls.yaml_tag,
                                            f'{node.name}-{node.age}')

        @classmethod
        def from_yaml(cls, _constructor, node):
            """from_yaml"""
            return cls(*node.value.split('-'))

    with open(BASE_DIR / f_yml, 'w', encoding='utf-8') as file_stream:
        yaml=YAML()
        yaml.default_flow_style = False
        yaml.dump([User('Anthon', 18)], file_stream)


if demo_decorator_class() is None:
            print("demo_decorator_class() = Pass")

demo_decorator_class() = Pass


---

## [Details](https://yaml.readthedocs.io/en/latest/detail.html)

In [7]:
INP_A = """\
abc:
  - a     # comment 1
xyz:
  a: 1    # comment 2
  b: 2
  c: 3
  d: 4
  f: 6    # comment 3 test OrderDict
  e: 5
"""

def demo_roundtrip_write():
    """
    demo_one
    round trip including comments
    ref: https://yaml.readthedocs.io/en/latest/detail.html
    """
    yaml = ruamel.yaml.YAML()  # defaults to round-trip
    data = yaml.load(INP_A)

    data['abc'].append('b')
    data['abc'].yaml_add_eol_comment('comment 4', 1)    # takes column of comment 1

    data['xyz'].yaml_add_eol_comment('comment 5', 'c')  # takes column of comment 2
    data['xyz'].yaml_add_eol_comment('comment 6', 'e')  # takes column of comment 3
    data['xyz'].yaml_add_eol_comment('comment 7', 'd', column=20)  # 20=space, 21=#, 22=msg

    yaml.dump(data, sys.stdout)


if demo_roundtrip_write() is None:
            print("demo_roundtrip_write() = Pass")

OUT_A = """\
abc:
- a       # comment 1
- b       # comment 4
xyz:
  a: 1    # comment 2
  b: 2
  c: 3    # comment 5
  d: 4              # comment 7
  f: 6    # comment 3 test OrderDict
  e: 5    # comment 6
"""

abc:
- a       # comment 1
- b       # comment 4
xyz:
  a: 1    # comment 2
  b: 2
  c: 3    # comment 5
  d: 4              # comment 7
  f: 6    # comment 3 test OrderDict
  e: 5    # comment 6
demo_roundtrip_write() = Pass


---

## [Examples](https://yaml.readthedocs.io/en/latest/example.html)

    demo_output_as_string()

In [8]:
INP_B = """\
# example
name:
  # details
  family: Smith   # very common
  given: Alice    # one of the siblings
"""

def demo_basic():
    """demo_basic"""
    yaml = YAML()
    code = yaml.load(INP_B)
    code['name']['given'] = 'Bob'

    yaml.dump(code, sys.stdout)


if demo_basic() is None:
            print("demo_basic() = Pass")

# example
name:
  # details
  family: Smith   # very common
  given: Bob      # one of the siblings
demo_basic() = Pass


YAML handcrafted anchors and references as well as key merging are preserved. The merged keys can transparently be accessed using [] and .get():

In [9]:
INP_C = """\
- &CENTER {x: 1, y: 2}
- &LEFT {x: 0, y: 2}
- &BIG {r: 10}
- &SMALL {r: 1}
# All the following maps are equal:
# Explicit keys
- x: 1
  y: 2
  r: 10
  label: center/big
# Merge one map
- <<: *CENTER
  r: 10
  label: center/big
# Merge multiple maps
- <<: [*CENTER, *BIG]
  label: center/big
# Override
- <<: [*BIG, *LEFT, *SMALL]
  x: 1
  label: center/big
"""

def demo_anchors():
    """demo_anchors"""
    yaml = YAML()
    data = yaml.load(INP_C)
    assert data[7]['y'] == 2
    yaml.dump(data, sys.stdout)


if demo_anchors() is None:
            print("demo_anchors() = Pass")

- &CENTER {x: 1, y: 2}
- &LEFT {x: 0, y: 2}
- &BIG {r: 10}
- &SMALL {r: 1}
# All the following maps are equal:
# Explicit keys
- x: 1
  y: 2
  r: 10
  label: center/big
# Merge one map
- <<: *CENTER
  r: 10
  label: center/big
# Merge multiple maps
- <<: [*CENTER, *BIG]
  label: center/big
# Override
- <<: [*BIG, *LEFT, *SMALL]
  x: 1
  label: center/big
demo_anchors() = Pass


In [10]:
INP_D = """\
first_name: Art
occupation: Architect  # This is an occupation comment
about: Art Vandelay is a fictional character that George invents...
"""

def demo_comments():
    """demo_comments"""
    yaml = YAML()
    data = yaml.load(INP_D)
    data.insert(1, 'last name', 'Vandelay', comment="new key")
    yaml.dump(data, sys.stdout)


if demo_comments() is None:
            print("demo_comments() = Pass")

first_name: Art
last name: Vandelay    # new key
occupation: Architect  # This is an occupation comment
about: Art Vandelay is a fictional character that George invents...
demo_comments() = Pass


In [11]:
def demo_compact():
    """demo_compact"""
    dict_content = [dict(b=2), [3, 4]]
    yaml = YAML()
    yaml.dump(dict_content, sys.stdout)
    print('='*15)
    yaml = YAML()
    yaml.compact(seq_seq=False, seq_map=False)
    yaml.dump(dict_content, sys.stdout)


if demo_compact() is None:
            print("demo_compact() = Pass")

- b: 2
- - 3
  - 4
-
  b: 2
-
  - 3
  - 4
demo_compact() = Pass


In [12]:
def demo_three_streams():
    """demo_three_streams"""
    data = {1: {1: [{1: 1, 2: 2}, {1: 1, 2: 2}], 2: 2}, 2: 42}

    yaml = YAML()
    yaml.explicit_start = True
    print("\n### 1st output ###")
    yaml.dump(data, sys.stdout)

    yaml.indent(sequence=4, offset=2)
    print("...\n### 2nd output ###")
    yaml.dump(data, sys.stdout)

    def sequence_indent_four(file_stream):
        # this will fail on directly nested lists: {1; [[2, 3], 4]}
        levels = []
        ret_val = ''
        for line in file_stream.splitlines(True):
            line_strip = line.lstrip()
            indent = len(line) - len(line_strip)
            if line_strip.startswith('- '):
                if not levels or indent > levels[-1]:
                    levels.append(indent)
                elif levels:
                    if indent < levels[-1]:
                        levels = levels[:-1]
                # same -> do nothing
            else:
                if levels:
                    if indent <= levels[-1]:
                        while levels and indent <= levels[-1]:
                            levels = levels[:-1]
            ret_val += '  ' * len(levels) + line
        return ret_val

    yaml = YAML()
    yaml.explicit_start = True
    print("...\n### 3rd output ###")
    yaml.dump(data, sys.stdout, transform=sequence_indent_four)
    print("...")


if demo_three_streams() is None:
            print("demo_three_streams() = Pass")


### 1st output ###
---
1:
  1:
  - 1: 1
    2: 2
  - 1: 1
    2: 2
  2: 2
2: 42
...
### 2nd output ###
---
1:
  1:
    - 1: 1
      2: 2
    - 1: 1
      2: 2
  2: 2
2: 42
...
### 3rd output ###
---
1:
  1:
    - 1: 1
      2: 2
    - 1: 1
      2: 2
  2: 2
2: 42
...
demo_three_streams() = Pass


In [13]:
def demo_output_as_string():
    """demo_output_as_string"""
    class MyYAML(YAML):
        """MyYAML
        This class has a dump method to handle
        """
        def dump(self, data, stream=None, **kw):
            inefficient = False
            if stream is None:
                inefficient = True
                stream = StringIO()
            YAML.dump(self, data, stream, **kw)
            if inefficient:
                return stream.getvalue()

    yaml = MyYAML()   # or typ='safe'/'unsafe' etc

    print(yaml.dump(dict(a=1, b=2)))

    # instead of
    # yaml.dump((dict(a=1, b=2)), sys.stdout)
    # print()  # or sys.stdout.write('\n')


if demo_output_as_string() is None:
            print("demo_output_as_string() = Pass")

a: 1
b: 2

demo_output_as_string() = Pass


---

## [Departure from previous API](https://yaml.readthedocs.io/en/latest/api.html)

__WIP__


In [14]:
from pathlib import Path
from ruamel.yaml import YAML

yaml = YAML(typ='safe')
yaml.default_flow_style = False
data = yaml.load("abc: 1")
out = Path('/tmp/out.yaml')
yaml.dump(data, out)

### Loading

```python
yaml = ruamel.yaml.YAML()
yaml.allow_duplicate_keys = True
yaml.load(stream)
```

### Dumping a multi-documents YAML stream

In [15]:
MULTI_DOC = """\
abc:
  - a     # comment 1

---

xyz:
  a: 1    # comment 2
  b: 2
  c: 3
  d: 4
  f: 6    # comment 3 test OrderDict
  e: 5
"""

In [16]:
yaml = YAML()
data = list(yaml.load_all(MULTI_DOC))
# do something on data[0], data[1], etc.
yaml.dump_all(data, sys.stdout)

abc:
- a       # comment 1

---

xyz:
  a: 1    # comment 2
  b: 2
  c: 3
  d: 4
  f: 6    # comment 3 test OrderDict
  e: 5


with YAML(output=sys.stdout) as yaml:
        yaml.explicit_start = True
        for data in yaml.load_all(Path(multi_document_filename)):
            # do something on data
            yaml.dump(data)

list_of_filenames = ['x.yaml', 'y.yaml', ]
with YAML(output=sys.stdout) as yaml:
        yaml.explicit_start = True
        for path in list_of_filenames:
            with open(path) as fp:
                yaml.dump(yaml.load(fp))

### Dumping

# in myyaml.py
if ruamel.yaml.version_info < (0, 15):
    class MyYAML(yaml.YAML):
        def __init__(self):
            yaml.YAML.__init__(self)
            self.preserve_quotes = True
            self.indent(mapping=4, sequence=4, offset=2)
# in your code
try:
    from myyaml import MyYAML
except (ModuleNotFoundError, ImportError):
    if ruamel.yaml.version_info >= (0, 15):
        raise

# some pathlib.Path
from pathlib import Path
inf = Path('/tmp/in.yaml')
outf = Path('/tmp/out.yaml')

if ruamel.yaml.version_info < (0, 15):
    with inf.open() as ifp:
         data = yaml.round_trip_load(ifp, preserve_quotes=True)
    with outf.open('w') as ofp:
         yaml.round_trip_dump(data, ofp, indent=4, block_seq_indent=2)
else:
    yml = MyYAML()
    # no need for with statement when using pathlib.Path instances
    data = yml.load(inf)
    yml.dump(data, outf)